In [ ]:
#!pip install deepface
#!pip install tf-keras
#!pip install scikit-learn
#!pip install matplotlib

In [ ]:
from deepface import DeepFace
from PIL import Image, ImageDraw
from IPython.display import display
import os
import shutil
from sklearn.model_selection import train_test_split

Define the folder and image name

In [ ]:
folder_name = "../dataset/images/original/"
modified_folder_name = "../dataset/images/processed/"
cropped_folder_name = "../dataset/images/cropped/"

# Create the processed and cropped folders if they don't exist
os.makedirs(modified_folder_name, exist_ok=True)
os.makedirs(cropped_folder_name, exist_ok=True)

for image_name in os.listdir(folder_name):
    # Only process if the file is an image (e.g., PNG, JPG)
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load image
        image_path = os.path.join(folder_name, image_name)
        image = Image.open(image_path)

        # Analyze the image
        analysis = DeepFace.analyze(img_path=image_path, detector_backend='retinaface', enforce_detection=False)
        
        # Draw bounding boxes for each face detected
        draw = ImageDraw.Draw(image)
        for idx, item in enumerate(analysis):
            region = item['region']
            x, y, w, h = region['x'], region['y'], region['w'], region['h']

            # Draw rectangle on the original image
            draw.rectangle([(x, y), (x + w, y + h)], outline='green', width=4)

            # Crop the detected face
            cropped_face = image.crop((x, y, x + w, y + h))
            cropped_face_path = os.path.join(cropped_folder_name, f"{image_name}_face_{idx + 1}.png")
            cropped_face.save(cropped_face_path)
            print(f"Cropped face saved as {cropped_face_path}")

        # Save the image with bounding boxes
        modified_image_path = os.path.join(modified_folder_name, "MOD_" + image_name)
        image.save(modified_image_path)
        
        # Optionally, display the processed image
        modified_image = Image.open(modified_image_path)
        display(modified_image)


## NN creation and training

In [ ]:
# Paths to the original dataset
dataset_dir = '../dataset'  # Original dataset directory
mask_dir = os.path.join(dataset_dir, 'mask')
no_mask_dir = os.path.join(dataset_dir, 'no_mask')

# Paths for training and validation directories
train_dir = 'dataset/train'
val_dir = 'dataset/val'

# Create directories for train/val split
os.makedirs(os.path.join(train_dir, 'mask'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'no_mask'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'mask'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'no_mask'), exist_ok=True)

# Function to split and create symlinks
def split_and_link_files(source_dir, train_class_dir, val_class_dir, test_size=0.2):
    # Get all files in the source directory
    files = os.listdir(source_dir)
    
    # Split into training and validation sets
    train_files, val_files = train_test_split(files, test_size=test_size, random_state=42)
    
    # Create symlinks for training files
    for file in train_files:
        source_file = os.path.join(source_dir, file)
        target_file = os.path.join(train_class_dir, file)
        os.symlink(source_file, target_file)  # Create a symbolic link
    
    # Create symlinks for validation files
    for file in val_files:
        source_file = os.path.join(source_dir, file)
        target_file = os.path.join(val_class_dir, file)
        os.symlink(source_file, target_file)  # Create a symbolic link
    
    # Return the counts of training and validation files
    return len(train_files), len(val_files)

# Split and link the "mask" images
mask_train_count, mask_val_count = split_and_link_files(mask_dir, os.path.join(train_dir, 'mask'), os.path.join(val_dir, 'mask'))

# Split and link the "no_mask" images
no_mask_train_count, no_mask_val_count = split_and_link_files(no_mask_dir, os.path.join(train_dir, 'no_mask'), os.path.join(val_dir, 'no_mask'))

# Print the quantities for each class
print("Dataset Quantities:")
print(f"Mask - Train: {mask_train_count}, Validation: {mask_val_count}")
print(f"No Mask - Train: {no_mask_train_count}, Validation: {no_mask_val_count}")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Paths to the training and validation datasets
train_dir = 'dataset/train'
val_dir = 'dataset/val'

# Image size (resize all images to the same size)
image_size = (128, 128)

# Rescale the pixel values for training set (without augmentation)
train_datagen = ImageDataGenerator(rescale=1./255)

# Rescale the pixel values for validation set (no augmentation)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load the training dataset from the directory
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Directory for the training images
    target_size=image_size,  # Resize the images
    batch_size=32,
    class_mode='binary',  # Binary classification (mask/no mask)
    shuffle=True  # Shuffle training images
)

# Load the validation dataset from the directory
validation_generator = validation_datagen.flow_from_directory(
    val_dir,  # Directory for the validation images
    target_size=image_size,  # Resize the images
    batch_size=32,
    class_mode='binary',  # Binary classification (mask/no mask)
    shuffle=False  # Do not shuffle validation images
)

# Build the CNN model
model = models.Sequential([
    # Convolutional Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional Layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional Layer 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten the output to feed into fully connected layers
    layers.Flatten(),
    
    # Dense Layer 1
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    
    # Output Layer: Sigmoid activation for binary classification
    layers.Dense(1, activation='sigmoid')
])

# Compile the model with binary crossentropy loss (for binary classification) and Adam optimizer
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Number of epochs to train the model
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

i=1
# Save the trained model
model.save('face_mask_classifier_no_augmentatio_'+i+'.h5')
i+=1
# Plot accuracy and loss curves
plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


